In [23]:
import csv
import json
import os,glob
from collections import defaultdict
import requests
from io import BytesIO
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

def read_car_info(filepath):
    car_info = []
    with open(filepath, 'r') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',')
        for row in spamreader:
            if row[6]=='' or row[5]=='' or row[2]=='' or row[3]=='' or row[1]=='' or row[4]=='':
                continue
            car_info.append({"car_id":row[1], "pp_brand_id":row[2], "pp_genre_id":row[3], "left_behind":row[5], "left_front": row[6], 'chinese':row[4]})
    return car_info
def read_car_pic(code):
    url = 'http://icdn.startcarlife.com/img/'+ code[:4]+'/'+code[4]+'/'+code+'.jpg'
    response = requests.get(url)
    try:
        img = Image.open(BytesIO(response.content))
    except:
        print (code," failed")
        img = None
    return img

def load_bad_item_from_box(bb_car):
    bad_item = []
    for item in bb_car:
        if len(bb_car[item]) == 1 and bb_car[item][0] == 'error':
            bad_item.append(item)
    bad_item.sort()
    bad_item = [item.split('.')[0] for item in bad_item]
    bad_item = [item.split('_')[0] for item in bad_item]
    bad_item = [item.split('/')[-1] for item in bad_item]
    bad_item = list(set(bad_item))
    return bad_item




## from csv to json

In [2]:
#audi_info = read_car_info('./audi.csv')
#bmw_info = read_car_info('./bmw.csv') #
#benz_info = read_car_info('./benz.csv') #48
# 
#info_list = glob.glob('./car_list/*.csv')
#info_list.remove('./car_list/all.csv')
info_list = ['./car_list/all.csv']
car_info = []
for item in info_list:
    car_info += read_car_info(item)
with open('allcar.json', 'w') as f:
    for item in car_info:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

In [4]:
print len(car_info),len(bad_item)

29098 155


In [29]:
from io import open
car_info = [json.loads(line) for line in open('allcar.json', encoding='gbk')]
car_info[0]


{u'car_id': u'341233',
 u'chinese': u'\u798f\u7279',
 u'left_behind': u'1505597222f3a691',
 u'left_front': u'150574721a35e421',
 u'pp_brand_id': u'53',
 u'pp_genre_id': u'1340'}

In [34]:
for item in car_info:
    if item['left_front'] == '15046462e556fe61':
        print item
        break

{u'pp_brand_id': u'12', u'left_front': u'15046462e556fe61', u'chinese': u'\u65e5\u4ea7', u'pp_genre_id': u'60', u'left_behind': u'15044763251ad801', u'car_id': u'432582'}


## from json to jpg

In [6]:
from io import open
car_info = [json.loads(line) for line in open('allcar.json', encoding='gbk')]
car_info[0]
i = 0
#bad_item
with open('./bball.npy') as f:
    bb_car = np.load(f).item()
bad_item = load_bad_item_from_box(bb_car)

for item in car_info:
    path = os.path.join('../',item['pp_brand_id'],item['pp_genre_id'])
    if not os.path.exists(path):
        os.makedirs(path)
        print (path)
    if not os.path.exists(os.path.join(path,item['car_id']+'_b.jpg')):
        if item['car_id'] in bad_item:
            #print item['car_id'], ': no car detected'
            continue
        img = read_car_pic(item['left_front'])
        img2 = read_car_pic(item['left_behind'])
        if img and img2:
            img.save(os.path.join(path,item['car_id']+'.jpg'))
            img2.save(os.path.join(path,item['car_id']+'_b.jpg'))
            i += 1
            if i % 50 == 0:
                print('save ',i,' pic')
            

(u'1507654ec79b60f1', ' failed')
(u'150725b8adf0f821', ' failed')
(u'1507154a6e788931', ' failed')
(u'15083052d10b2621', ' failed')
(u'15095392aa193ee1', ' failed')
(u'150926cef061d871', ' failed')
(u'150985c4389dd441', ' failed')
(u'150989c2a46bca31', ' failed')
(u'150966c858db0541', ' failed')
(u'151080960571f5a1', ' failed')
(u'151299a0a8c0aef1', ' failed')
(u'160195f10dc7bd51', ' failed')
(u'1602852aaa73df91', ' failed')
(u'1603536222f199e1', ' failed')
(u'1603246229620a21', ' failed')
(u'160661987b196c01', ' failed')
(u'160651989adca7c1', ' failed')
(u'160694da45947251', ' failed')
(u'160679da4ad7d811', ' failed')
(u'1607236afc3c9731', ' failed')
(u'1607956b003e9b01', ' failed')


## detect box in pic and find out bad item

In [18]:
#filter jpg without car
import numpy as np
import os
import scipy.io
import tensorflow as tf
from collections import defaultdict
from matplotlib import pyplot as plt
import PIL.Image as pil_image
from keras.models import load_model
import json, glob

def load_image_into_numpy_array(image):
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape(
        (im_height, im_width, 3)).astype(np.uint8)

data_dir = '/root/playground/START/car_class/yiche_image/yiche_processed/'
TEST_IMAGE_PATHS = []
bb_car = defaultdict(list)

makes = os.listdir(data_dir)
try:
    makes.remove('bba')
except:
    pass
for make in makes:
    models = os.listdir(os.path.join(data_dir, make))
    for model in models:
        #pics = glob.glob(os.path.join(data_dir, make, model))
        pics = glob.glob(os.path.join(data_dir, make, model)+'/*.jpg')
        for pic in pics:
            TEST_IMAGE_PATHS.append(pic)
TEST_IMAGE_PATHS.sort()

detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile('../../model/ssd_mobilenet_v1_coco_11_06_2017/frozen_inference_graph.pb', 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [21]:
for item in TEST_IMAGE_PATHS:
    print '/'.join(item.split('/')[-3:])
    break
    if '/'.join(item.split('/')[-3:]) == '20036/3095/bb119902.jpg':
        print item

10000/1906/bl2684.jpg


In [23]:
#bad_item
bb_car = np.load('../../yiche_image/map/yiche_box.npy').item()

In [25]:
j = 0
with detection_graph.as_default():
    with tf.Session(graph=detection_graph) as sess:
        for image_path in TEST_IMAGE_PATHS:
            j+=1
            if j % 20000 == 0:
                np.save('./tmpbox.npy',bb_car)
                print j, 'image done'
            if len(bb_car['/'.join(image_path.split('/')[-3:])]) > 0:
                continue
            try:
                image = pil_image.open(image_path)
                image_np = load_image_into_numpy_array(image)
            except:
                print 'image size error', image_path
                bb_car['/'.join(image_path.split('/')[-3:])].append('error')
                continue
            image_np_expanded = np.expand_dims(image_np, axis=0)
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            scores = detection_graph.get_tensor_by_name('detection_scores:0')
            classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            # Actual detection.
            (boxes, scores, classes, num_detections) = sess.run(
                [boxes, scores, classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # save bb
            for i in range(11):
                if scores[0][i] < 0.5:
                    if len(bb_car['/'.join(image_path.split('/')[-3:])]) == 0:
                        print 'no car detected:', image_path
                        bb_car['/'.join(image_path.split('/')[-3:])].append('error')
                    break
                elif classes[0][i] == 8. or classes[0][i] == 3. or classes[0][i] == 6.:
                    bb_car['/'.join(image_path.split('/')[-3:])].append(boxes[0][i])

20000 image done
40000 image done


In [4]:
np.save('/root/playground/START/car_class/start/bba/yiche_box.npy', bb_car)

## delete bad item

In [28]:
path = '/root/playground/START/car_class/yiche_image/yiche_processed/'
path = '../'
makes = os.listdir(path)
makes.remove('bba')
bb_car = np.load('./bball.npy').item()
bad_item = load_bad_item_from_box(bb_car)
for make in makes:
    models = os.listdir(os.path.join(path, make))
    for model in models:
        pics = glob.glob(os.path.join(path, make, model)+'/*.jpg')
        for pic in pics:
            car_id = pic.split('/')[-1].split('_')[0].split('.')[0]
            if len(bb_car['/'.join(pic.split('/')[-3:])])==1 and bb_car['/'.join(pic.split('/')[-3:])][0] == 'error':
                os.remove(pic)
                print pic

/root/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:34: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
/root/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  del sys.path[0]


../115/996/620831.jpg
../115/998/680055.jpg
../115/998/639327.jpg
../47/299/724206.jpg
../130/2413/542392.jpg
../81/760/245123.jpg
../132/1126/501071.jpg
../132/1134/742163.jpg
../132/1134/533420.jpg
../38/244/552241.jpg
../38/244/27212.jpg
../39/356/805534.jpg
../139/1256/472379.jpg
../96/865/712337.jpg
../96/2357/723250.jpg
../96/2357/660685.jpg
../140/1261/738181.jpg
../44/270/529675.jpg
../44/271/741031.jpg
../44/271/727072.jpg
../44/273/772127.jpg
../44/273/568308.jpg
../44/273/671178.jpg
../44/273/734747.jpg
../44/273/772173.jpg
../40/2390/230982.jpg
../98/867/657926.jpg
../97/20209/792496.jpg
../107/2359/134575.jpg
../107/1154/439979.jpg
../73/697/524797.jpg
../73/697/785813.jpg
../73/697/731301.jpg
../73/697/712535.jpg
../73/699/759659.jpg
../73/699/530868.jpg
../73/699/759652.jpg
../73/699/462834.jpg
../73/1334/767431.jpg
../28/197/773016.jpg
../28/201/225354.jpg
../28/201/706654.jpg
../28/2545/673280.jpg
../28/203/734800.jpg
../28/203/279792.jpg
../28/20044/770264.jpg
../28/2